# Day 7: Bridge Repair

The Historians take you to a familiar [rope bridge](https://adventofcode.com/2022/day/9) over a river in the middle of a jungle. The Chief isn't on this side of the bridge, though; maybe he's on the other side?

When you go to cross the bridge, you notice a group of engineers trying to repair it. (Apparently, it breaks pretty frequently.) You won't be able to cross until it's fixed.

You ask how long it'll take; the engineers tell you that it only needs final calibrations, but some young elephants were playing nearby and _stole all the operators_ from their calibration equations! They could finish the calibrations if only someone could determine which test values could possibly be produced by placing any combination of operators into their calibration equations (your puzzle input).

For example:

```
190: 10 19
3267: 81 40 27
83: 17 5
156: 15 6
7290: 6 8 6 15
161011: 16 10 13
192: 17 8 14
21037: 9 7 18 13
292: 11 6 16 20
```

Each line represents a single equation. The test value appears before the colon on each line; it is your job to determine whether the remaining numbers can be combined with operators to produce the test value.

Operators are _always evaluated left-to-right_, _not_ according to precedence rules. Furthermore, numbers in the equations cannot be rearranged. Glancing into the jungle, you can see elephants holding two different types of operators: _add_ (`+`) and _multiply_ (`*`).

Only three of the above equations can be made true by inserting operators:

- `190: 10 19` has only one position that accepts an operator: between `10` and `19`. Choosing `+` would give `29`, but choosing `*` would give the test value (`10 * 19 = 190`).
- `3267: 81 40 27` has two positions for operators. Of the four possible configurations of the operators, _two_ cause the right side to match the test value: `81 + 40 * 27` and `81 * 40 + 27` both equal `3267` (when evaluated left-to-right)!
- `292: 11 6 16 20` can be solved in exactly one way: `11 + 6 * 16 + 20`.

The engineers just need the _total calibration result_, which is the sum of the test values from just the equations that could possibly be true. In the above example, the sum of the test values for the three equations listed above is `_3749_`.

Determine which equations could possibly be true. _What is their total calibration result?_

In [1]:
$sample_data = (@'
190: 10 19
3267: 81 40 27
83: 17 5
156: 15 6
7290: 6 8 6 15
161011: 16 10 13
192: 17 8 14
21037: 9 7 18 13
292: 11 6 16 20
'@).Replace("`r", '') -split "`n" 

$input_data = Get-Content .\data.txt

In [ ]:
function dostuff {
    [CmdletBinding()] Param(
        [Parameter(Mandatory=$false)]
        [switch] $sample,
        [Parameter(Mandatory=$false)]
        [switch] $part2
    )
    if($sample) {
        $data = $sample_data
        $out = "With *SAMPLE* dataset, part "
    } else {
        $data = $input_data
        $out = "Part "
    }
    $out += $part2 ? '2: ' : '1: '

    $dataLines = $data.Count
    $progress = 0
    $calibrationSum =0
    foreach($line in $data) {
        $temp = $line.split(': ')
        $calibration = [bigint]$temp[0]
        $operands = $temp[1].split(' ')

        Write-Debug "Testing $line..."
        $lefthandOperands = @($operands[0])
        for($i=1; $i -lt $operands.Count; $i++){
            $results = [System.Collections.ArrayList]::new()
            foreach($lefthandOperand in $lefthandOperands) {
                $results.add([bigint]$lefthandOperand * [bigint]$operands[$i]) | Out-Null
                $results.add([bigint]$lefthandOperand + [bigint]$operands[$i]) | Out-Null
                if($part2) {
                    $results.add([bigint]([string]$lefthandOperand + [string]$operands[$i])) | Out-Null
                }
            }
            Write-Debug "Stage $i, results of $($operands[$i]) */+(/||) [$($lefthandOperands -join ', ')]: [$($results -join ', ')]"
            #$lefthandOperands.add( $results | ? {$_ -lt $calibration} ) # only keep results still less than the calibration
            $lefthandOperands = $results.Clone()
        }
        
        if($calibration -in $results) {
            Write-Debug "Calibration successful!"
            $calibrationSum += $calibration
        }

        Write-Progress "Processing calibrations..." -PercentComplete ($progress++/$dataLines)

    }

    $out += "total calibration result is $calibrationSum"
    Write-Output $out
}

dostuff -sample -Debug
dostuff
dostuff -part2 -sample -Debug
dostuff -part2

DEBUG: Testing 190: 10 19...
DEBUG: Stage 1, results of 19 */+(/||) [10]: [190, 29]
DEBUG: Calibration successful!


                                                                                                    
 Processing calibrations...                                                                         
    Processing                                                                                      
    [                                                                                        ]      
                                                                                                    

DEBUG: Testing 3267: 81 40 27...
DEBUG: Stage 1, results of 40 */+(/||) [81]: [3240, 121]
DEBUG: Stage 2, results of 27 */+(/||) [3240, 121]: [87480, 3267, 3267, 148]
DEBUG: Calibration successful!
DEBUG: Testing 83: 17 5...
DEBUG: Stage 1, results of 5 */+(/||) [17]: [85, 22]
DEBUG: Testing 156: 15 6...
DEBUG: Stage 1, results of 6 */+(/||) [15]: [90, 21]
DEBUG: Testing 7290: 6 8 6 15...
DEBUG: Stage 1, results of 8 */+(/||) [6]: [48, 14]
DEBUG: Stage 2, results of 6 */+(/||) [48, 14]: [288, 54, 84, 20]
DEBUG: Stage 3, results of 15 */+(/||) [288, 54, 84, 20]: [4320, 303, 810, 69, 1260, 99, 300, 35]
DEBUG: Testing 161011: 16 10 13...
DEBUG: Stage 1, results of 10 */+(/||) [16]: [160, 26]
DEBUG: Stage 2, results of 13 */+(/||) [160, 26]: [2080, 173, 338, 39]
DEBUG: Testing 192: 17 8 14...
DEBUG: Stage 1, results of 8 */+(/||) [17]: [136, 25]
DEBUG: Stage 2, results of 14 */+(/||) [136, 25]: [1904, 150, 350, 39]
DEBUG: Testing 21037: 9 7 18 13...
DEBUG: Stage 1, results of 7 */+(/||) [9

In [1]:
# Characterization

$greatestOperands = [pscustomobject] @{
    line = ''
    number = 0
}
# Operands: 0 1 2 3  4  5  6
# 2^ops:    0 1 2 4  8 16 32
# sum all:  0 1 3 7 15 31 63 ???... Aliens.

$recursionDepths = @(0, 1)
$recursionDepths += for($i=2; $i -lt 13; $i++) {
    ([System.Math]::Pow(2, $i)-1)
}

$totalRecursion = 0
foreach($line in $input_data) {
    $numOperands = (($line.split(': '))[1].split(' ')).count
    if($numOperands -gt $greatestOperands.number) {
        $greatestOperands.line = $line
        $greatestOperands.number = $numOperands
    }
    $totalRecursion += $recursionDepths[$numOperands]
}

$greatestOperands
Write-Output "Worst possible max recursion: $totalRecursion"


line number
---- ------
          0
Worst possible max recursion: 0

